In [1]:
import pandas as pd
import pickle
with open('train.pkl','rb') as f:
    train = pickle.load(f)
with open('test.pkl','rb') as f:
    test = pickle.load(f)
with open('df_train.pkl','rb') as f:
    df_train = pickle.load(f)
with open('df_test.pkl','rb') as f:
    df_test = pickle.load(f)

In [2]:
%%time
df_train.drop(['api','tid','index'],axis=1,inplace=True)
df_test.drop(['api','tid','index'],axis=1,inplace=True)

CPU times: total: 0 ns
Wall time: 14.9 ms


In [3]:
def get_apis(df):
    # 按照file_id进行分组
    group_fileid = df.groupby('file_id')    
    # 统计file_id 和对应的 api_sequence
    file_api = {}    
    # 计算每个file_id的api_sequence
    for file_id, file_group in group_fileid:
        # 针对file_id 按照线程tid 和 顺序index进行排序
        result = file_group.sort_values(['tid', 'index'], ascending=True)
        #得到api的调用序列
        api_sequence = ' '.join(result['api'])
        #print(api_sequence)
        file_api[file_id] = api_sequence
    return file_api

In [4]:
%%time
train_apis = get_apis(train)
test_apis=get_apis(test)

CPU times: total: 1min
Wall time: 1min 4s


In [5]:
%%time
temp = pd.DataFrame.from_dict(train_apis, orient='index', columns=['api'])
temp = temp.reset_index().rename(columns={'index': 'file_id'})
df_train = df_train.merge(temp, on='file_id', how='left')
 
temp = pd.DataFrame.from_dict(test_apis, orient='index', columns=['api'])
temp = temp.reset_index().rename(columns={'index': 'file_id'})
df_test = df_test.merge(temp, on='file_id', how='left')
 
df_all = pd.concat([df_train, df_test], axis=0)
df_all.describe()

CPU times: total: 219 ms
Wall time: 286 ms


file_id         label      api_count   api_nunique       api_max  \
count  26842.000000  13887.000000   26842.000000  26842.000000  26842.000000   
mean    6719.090157      2.930655    6299.644885     49.188473    275.538857   
std     3888.360300      2.596575   17774.836337     30.210753     17.967609   
min        1.000000      0.000000       1.000000      1.000000     95.000000   
25%     3356.000000      0.000000     228.000000     24.000000    261.000000   
50%     6711.000000      3.000000    1812.000000     47.000000    281.000000   
75%    10066.000000      5.000000    7311.500000     71.000000    286.000000   
max    13887.000000      7.000000  888204.000000    175.000000    300.000000   

            api_min       api_std      api_mean    api_median       api_ptp  \
count  26842.000000  26807.000000  26842.000000  26842.000000  26842.000000   
mean      31.238581     49.062198    166.249683    165.519540    244.300276   
std       52.492967     15.670388     34.107152     44.295587     56.175952   
min        0.000000      0.000000     33.525295      6.000000      0.000000   
25%        6.000000     40.854431    147.546273    137.000000    241.000000   
50%        9.000000     49.604570    160.940639    152.000000    261.000000   
75%       25.000000     59.675767    181.213417    181.000000    275.000000   
max      281.000000    114.551299    299.639072    300.000000    300.000000   

       ...    tid_median       tid_ptp    index_count  index_nunique  \
count  ...  26842.000000  26842.000000   26842.000000   26842.000000   
mean   ...   2535.665524    686.551524    6299.644885    1779.856978   
std    ...    227.619883    854.599441   17774.836337    1934.252149   
min    ...    184.000000      0.000000       1.000000       1.000000   
25%    ...   2428.000000    100.000000     228.000000     125.000000   
50%    ...   2540.000000    384.000000    1812.000000     961.000000   
75%    ...   2676.000000    792.000000    7311.500000    3661.750000   
max    ...   5972.000000  20372.000000  888204.000000    5001.000000   

          index_max  index_min     index_std    index_mean  index_median  \
count  26842.000000    26842.0  26807.000000  26842.000000  26842.000000   
mean    1778.856978        0.0    510.268837    773.072243    720.564414   
std     1934.252149        0.0    555.853917    845.683594    807.020543   
min        0.000000        0.0      0.000000      0.000000      0.000000   
25%      124.000000        0.0     37.094474     61.407468     60.500000   
50%      960.000000        0.0    277.849780    459.500000    412.000000   
75%     3660.750000        0.0   1003.867770   1333.080881   1089.000000   
max     5000.000000        0.0   1654.531360   2500.000000   2500.000000   

          index_ptp  
count  26842.000000  
mean    1778.856978  
std     1934.252149  
min        0.000000  
25%      124.000000  
50%      960.000000  
75%     3660.750000  
max     5000.000000  

[8 rows x 26 columns]

In [6]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vec=TfidfVectorizer(ngram_range=(1,4),min_df=0.01,max_df=0.99)
api_features = vec.fit_transform(df_all['api'])

#降维
from sklearn.decomposition import PCA
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
#执行映射，把维度降为500
transformer = GaussianRandomProjection(n_components=500)
api_features = transformer.fit_transform(api_features)

CPU times: total: 5min 14s
Wall time: 5min 17s


In [7]:
df_apis = pd.DataFrame(api_features)

df_train_apis=df_apis[df_apis.index<=13886]
df_test_apis=df_apis[df_apis.index>13886]
df_test_apis.index=range(len(df_test_apis))
df_train_apis.head(1)

0         1         2         3         4         5         6    \
0 -0.012461  0.000844  0.030476 -0.033235 -0.002896 -0.056077 -0.033446   

        7         8         9    ...       490       491       492      493  \
0  0.025968  0.020923 -0.052141  ...  0.000891  0.001875  0.099369 -0.01022   

        494       495       496       497       498       499  
0  0.053438  0.060325 -0.037909  0.074527  0.045729  0.042371  

[1 rows x 500 columns]

In [8]:
#合并特征
df_train.drop(['api'],axis=1,inplace=True)
df_test.drop(['api'],axis=1,inplace=True)
df_train =df_train.merge(df_train_apis,left_index=True,right_index=True)
df_test =df_test.merge(df_test_apis,left_index=True,right_index=True)

In [9]:
%%time
import lightgbm as lgb
clf = lgb.LGBMClassifier(
           num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='multiclass',
            max_depth=-1, learning_rate=0.005, min_child_samples=3, random_state=2021,
            n_estimators=2000, subsample=1, colsample_bytree=1)
clf.fit(df_train.drop(['label'],axis=1),df_train['label'])
result_lgb=clf.predict_proba(df_test)
result_lgb = pd.DataFrame(result_lgb,columns=['prob0','prob1','prob2','prob3','prob4','prob5','prob6','prob7'])
result_lgb['file_id']=df_test['file_id'].values

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022622 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 132851
[LightGBM] [Info] Number of data points in the train set: 13887, number of used features: 524
[LightGBM] [Info] Start training from score -1.025925
[LightGBM] [Info] Start training from score -3.320108
[LightGBM] [Info] Start training from score -2.451970
[LightGBM] [Info] Start training from score -2.829404
[LightGBM] [Info] Start training from score -4.933538
[LightGBM] [Info] Start training from score -1.174900
[LightGBM] [Info] Start training from score -3.294542
[LightGBM] [Info] Start training from score -2.234192
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

In [10]:
%%time
import xgboost as xgb
model_xgb = xgb.XGBClassifier(
    max_depth=9, learning_rate=0.005, n_estimators=2000, 
    objective='multi:softprob', tree_method="hist", device="cuda", 
    subsample=0.8, colsample_bytree=0.8, 
    min_child_weight=3, eval_metric='logloss', reg_lambda=0.5)
model_xgb.fit(df_train.drop(['label'],axis=1),df_train['label'])
result_xgb=model_xgb.predict_proba(df_test)
result_xgb = pd.DataFrame(result_xgb,columns=['prob0','prob1','prob2','prob3','prob4','prob5','prob6','prob7'])
result_xgb['file_id']=df_test['file_id'].values

F:\AnaConda\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:58:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


CPU times: total: 5min 49s
Wall time: 5min 36s


In [11]:
result = result_lgb.copy()
weight_lgb = 0.5
weight_xgb = 0.5
result['prob0'] = result_lgb['prob0']*weight_lgb+result_xgb['prob0']*weight_xgb
result['prob1'] = result_lgb['prob1']*weight_lgb+result_xgb['prob1']*weight_xgb
result['prob2'] = result_lgb['prob2']*weight_lgb+result_xgb['prob2']*weight_xgb
result['prob3'] = result_lgb['prob3']*weight_lgb+result_xgb['prob3']*weight_xgb
result['prob4'] = result_lgb['prob4']*weight_lgb+result_xgb['prob4']*weight_xgb
result['prob5'] = result_lgb['prob5']*weight_lgb+result_xgb['prob5']*weight_xgb
result['prob6'] = result_lgb['prob6']*weight_lgb+result_xgb['prob6']*weight_xgb
result['prob7'] = result_lgb['prob7']*weight_lgb+result_xgb['prob7']*weight_xgb
columns=['file_id','prob0','prob1','prob2','prob3','prob4','prob5','prob6','prob7']
result.to_csv('./baseline3.csv',columns=columns,index=False)